In [ ]:
# import modules
import sqlite3
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np

# Objektif
1. Menentukan 5 item penjualan terbesar per kota/negara

2. menentukan seller penjualan terbesar per kota/negara

3. menentukan customer pembelian terbesar untuk mendapatnya program prioritas

# Mengakses Dataset

In [ ]:
# importing file path
data = 'olist.db'

# create a SQL connection to SQLite database
con = sqlite3.connect(data)

# creating cursor
cur = con.cursor()

# reading table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type='table'")]
print(table_list)

In [ ]:
# Akses Dataset
query = """
SELECT
    A.ORDER_ID,
    A.ORDER_ITEM_ID,
    D.SELLER_ID,
    D.SELLER_CITY,
    D.SELLER_STATE,
    B.CUSTOMER_ID,
    E.CUSTOMER_CITY,
    E.CUSTOMER_STATE,
    C.PRODUCT_ID,
    F.PRODUCT_CATEGORY_NAME_ENGLISH AS product_category_name,
    A.PRICE,
    A.FREIGHT_VALUE
FROM
    OLIST_ORDER_ITEMS_DATASET A,
    OLIST_ORDER_DATASET B,
    OLIST_PRODUCTS_DATASET C,
    OLIST_SELLERS_DATASET D,
    OLIST_ORDER_CUSTOMER_DATASET E,
    PRODUCT_CATEGORY_NAME_TRANSLATION F
WHERE
A.ORDER_ID = B.ORDER_ID
AND B.CUSTOMER_ID = E.CUSTOMER_ID
AND A.PRODUCT_ID = C.PRODUCT_ID
AND A.SELLER_ID = D.SELLER_ID
AND C.PRODUCT_CATEGORY_NAME = F.PRODUCT_CATEGORY_NAME
AND B.ORDER_STATUS NOT IN ('CANCELED', 'UNAVAILABLE')
"""
# Dalam proses pemahaman data, ditemukan bahwa tidak semua order diproses sehingga order_status dalam penelitian ini mengeluarkan order status 'canceled' dan 'unavailable'

data = pd.read_sql_query(query, con)
data.head(5)

# Eksplorasi dan Pemrosesan Data

In [ ]:
# Identifikasi NaN
data.info()

In [ ]:
# Memeriksa Niilai NaN pada Data
data.isnull().sum().sort_values(ascending=False)/len(data)*100

In [ ]:
# Identifikasi outlier
data.describe(include="all").T

In [ ]:
sns.boxplot(data['price'])

In [ ]:
sns.boxplot(data['freight_value'])

In [ ]:
# IQR
Q1_freight_value = np.percentile(data['freight_value'], 25, interpolation= 'midpoint')
Q3_freight_value = np.percentile(data['freight_value'], 75, interpolation= 'midpoint')
IQR_freight_value = Q3_freight_value - Q1_freight_value

# Removing Outlier
data = data[(data['freight_value'] <= (Q3_freight_value+1.5*IQR_freight_value)) & (data['freight_value'] >= (Q1_freight_value-1.5*IQR_freight_value))]

In [ ]:
# IQR
Q1_price = np.percentile(data['price'], 25, interpolation= 'midpoint')
Q3_price = np.percentile(data['price'], 75, interpolation= 'midpoint')
IQR_price = Q3_price - Q1_price

# Removing Outlier
data = data[(data['price'] <= (Q3_price+1.5*IQR_price)) & (data['freight_value'] >= (Q1_price-1.5*IQR_price))]

In [ ]:
# Identifikasi Data Duplicate
data[data.duplicated(keep=False)].head(20) # Tidak terdapat duplikasi data

# Tujuan Analisis

1. Menentukan 5 item penjualan terbesar (berdasarkan total harga, jumlah item terjual dan berdasarkan negara)

In [ ]:
# mempersiapkan data
penjualan_berdasarkan_negara = data[['product_id', 'product_category_name', 'seller_state', 'customer_state', 'order_item_id', 'price']]
penjualan_berdasarkan_negara['total'] = penjualan_berdasarkan_negara['order_item_id'] * penjualan_berdasarkan_negara['price']

# Penjualan terbesar berdasarkan total nilai (price * jumlah item)
penjualan_berdasarkan_negara.groupby(['product_category_name'])['order_item_id','total'].sum().sort_values(by='total',ascending=False).head(5)

In [ ]:
# Penjualan terbesar berdasarkan jumlah item terbesar
penjualan_berdasarkan_negara.groupby(['product_category_name'])['order_item_id','total'].sum().sort_values(by='order_item_id',ascending=False).head(5)

In [ ]:
# Penjualan terbesar berdasarkan negara seller 
penjualan_berdasarkan_negara.groupby(['seller_state'])['order_item_id','total'].sum().sort_values(by='total',ascending=False).head(5)

In [ ]:
# Penjualan terbesar berdasarkan negara customer
penjualan_berdasarkan_negara.groupby(['customer_state'])['order_item_id','total'].sum().sort_values(by='total',ascending=False).head(5)

2. menentukan seller penjualan terbesar per kota/negara

In [ ]:
# mempersiapkan data
seller = data[['seller_id', 'seller_city', 'seller_state', 'product_category_name', 'order_item_id', 'price', 'freight_value']]
seller['total_price'] = seller['order_item_id'] * seller['price']


In [46]:
# top 5 seller dan negaranya
seller.groupby(['seller_id','seller_city', 'seller_state'])['seller_id','seller_city', 'seller_state','total_price'].sum().sort_values(by='total_price', ascending=False).head(5)

C:\Users\KomputerSKKK\AppData\Local\Temp\ipykernel_13532\1346930050.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  seller.groupby(['seller_id','seller_city', 'seller_state'])['seller_id','seller_city', 'seller_state','total_price'].sum().sort_values(by='total_price', ascending=False).head(5)


,,,total_price
seller_id,seller_city,seller_state,
4a3ca9315b744ce9f8e9374361493884,ibitinga,SP,190737.17
da8622b14eb17ae2831f4ac5b9dab84a,piracicaba,SP,167264.45
4869f7a5dfa277a7dca6462dcf3b52b2,guariba,SP,151140.33
1f50f920176fa81dab994f9023523100,sao jose do rio preto,SP,149775.84
7a67c85e85bb2ce8582c35f2203ad736,sao paulo,SP,136564.78


3. menentukan customer pembelian terbesar untuk mendapatnya program prioritas

In [ ]:
# mempersiapkan data
customer = data[['customer_id', 'customer_city', 'customer_state', 'product_category_name', 'order_item_id', 'price', 'freight_value']]
customer['total_price'] = customer['order_item_id'] * customer['price']

In [47]:
# top 5 customer dan negaranya
customer.groupby(['customer_id', 'customer_city', 'customer_state'])['customer_id', 'customer_city', 'customer_state','total_price'].sum().sort_values(by='total_price', ascending=False).head(5)

C:\Users\KomputerSKKK\AppData\Local\Temp\ipykernel_13532\1505513437.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  customer.groupby(['customer_id', 'customer_city', 'customer_state'])['customer_id', 'customer_city', 'customer_state','total_price'].sum().sort_values(by='total_price', ascending=False).head(5)


,,,total_price
customer_id,customer_city,customer_state,
be1b70680b9f9694d8c70f41fa3dc92b,sao paulo,SP,21000.0
bd5d39761aa56689a265d95d8d32b8be,goiania,GO,20727.0
91f92cfee46b79581b05aa974dd57ce5,curitiba,PR,8424.0
10de381f8a8d23fff822753305f71cae,uniao da vitoria,PR,7858.8
daf15f1b940cc6a72ba558f093dc00dd,celso ramos,SC,6232.2
